In [1]:
import numpy as np
import pandas as pd
import pypsa
import highspy

from helper_functions import grouped_gas_ramping, chp_unit_profile, apply_nuclear_outages

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
DATA_FILE = 'uk-2030-falling-short.xlsx'

In [3]:
# groups gas plants together to add in cold start constraints to CCGTs
add_gas_cold_start = False 
# load profile doesnt include H2 demand if set True
hydrogen_demand = True
# filter gas units on operating status 
filter_operating_ff_plants = False 

In [4]:
# input parameters
P_MIN_NUCLEAR = 0.4

In [6]:
load = pd.read_excel(DATA_FILE, sheet_name='load', parse_dates=['t'], index_col='t',usecols='A:AK').dropna(axis=1, how='all').dropna()
load_hydrogen = pd.read_excel(DATA_FILE, sheet_name='load_hydrogen', parse_dates=['t'], index_col='t',usecols='A:AK').dropna(axis=1, how='all').dropna()
pv = pd.read_excel(DATA_FILE, sheet_name='pv', parse_dates=['t'], index_col='t',usecols='A:AK').dropna(axis=1, how='all').dropna()
wind = pd.read_excel(DATA_FILE, sheet_name='wind', parse_dates=['t'], index_col='t',usecols='A:AK').dropna(axis=1, how='all').dropna()
wind_offshore = pd.read_excel(DATA_FILE, sheet_name='wind_offshore', parse_dates=['t'], index_col='t',usecols='A:AK').dropna(axis=1, how='all').dropna()
ror = pd.read_excel(DATA_FILE, sheet_name='ror', parse_dates=['t'], index_col='t',usecols='A:AK').dropna(axis=1, how='all').dropna()
inflow = pd.read_excel(DATA_FILE, sheet_name='inflow', parse_dates=['t'], index_col='t',usecols='A:AK').dropna(axis=1, how='all').dropna()
chp = pd.read_excel(DATA_FILE, sheet_name='chp', parse_dates=['t'], index_col='t').dropna(axis=1, how='all').dropna()
chp_bio = pd.read_excel(DATA_FILE, sheet_name='chp_bio', parse_dates=['t'], index_col='t').dropna(axis=1, how='all').dropna()

buses = pd.read_excel(DATA_FILE, sheet_name='buses', index_col=0, usecols='A:G')
cbf_buses = pd.read_excel(DATA_FILE, sheet_name='cbf_buses', index_col=0, usecols='A:F')
buses_hydrogen = pd.read_excel(DATA_FILE, sheet_name='buses_hydrogen', index_col=0, usecols='A:D')
lines = pd.read_excel(DATA_FILE, sheet_name='lines', index_col='name').dropna(axis=1, how='all').dropna()
links = pd.read_excel(DATA_FILE, sheet_name='links', index_col=0, usecols='A:H').dropna(axis=1, how='all').dropna()
links_electrolysis = pd.read_excel(DATA_FILE, sheet_name='links_electrolysis', index_col=0, usecols='A:I').dropna(axis=1, how='all').dropna()
gen_pv = pd.read_excel(DATA_FILE, sheet_name='gen_pv', index_col=0, usecols='A:E')
gen_wind = pd.read_excel(DATA_FILE, sheet_name='gen_wind', index_col=0, usecols='A:E')
gen_wind_offshore = pd.read_excel(DATA_FILE, sheet_name='gen_wind_offshore', index_col=0, usecols='A:E')
gen_gas = pd.read_excel(DATA_FILE, sheet_name='gen_gas', index_col=0, usecols='A:R')
gen_oil = pd.read_excel(DATA_FILE, sheet_name='gen_oil', index_col=0, usecols='A:J')
gen_coal = pd.read_excel(DATA_FILE, sheet_name='gen_coal', index_col=0,usecols='A:N')
gen_nuclear = pd.read_excel(DATA_FILE, sheet_name='gen_nuclear', index_col=0, usecols='A:I')
gen_biomass = pd.read_excel(DATA_FILE, sheet_name='gen_biomass', index_col=0, usecols='A:G')
gen_biogas = pd.read_excel(DATA_FILE, sheet_name='gen_biogas', index_col=0, usecols='A:G')
gen_ror = pd.read_excel(DATA_FILE, sheet_name='gen_ror', index_col=0, usecols='A:E')
gen_reservoir = pd.read_excel(DATA_FILE, sheet_name='gen_reservoir', index_col=0, usecols='A:E')
gen_other_res = pd.read_excel(DATA_FILE, sheet_name='gen_other_res', index_col=0, usecols='A:F')
gen_dsr = pd.read_excel(DATA_FILE, sheet_name='gen_dsr', index_col=0, usecols='A:F')
gen_gas_chp = pd.read_excel(DATA_FILE, sheet_name='gen_gas_chp', index_col=0, usecols='A:R')
gen_coal_chp = pd.read_excel(DATA_FILE, sheet_name='gen_coal_chp', index_col=0, usecols='A:R')
gen_oil_chp = pd.read_excel(DATA_FILE, sheet_name='gen_oil_chp', index_col=0, usecols='A:G')
gen_res_chp = pd.read_excel(DATA_FILE, sheet_name='gen_res_chp', index_col=0, usecols='A:G')
gen_bio_chp = pd.read_excel(DATA_FILE, sheet_name='gen_bio_chp', index_col=0, usecols='A:G')
gen_cbf = pd.read_excel(DATA_FILE, sheet_name='gen_cbf', index_col=0, usecols='A:E')
st_hps = pd.read_excel(DATA_FILE, sheet_name='st_hps', index_col=0, usecols='A:I')
st_battery = pd.read_excel(DATA_FILE, sheet_name='st_battery', index_col=0, usecols='A:I')
st_hydrogen = pd.read_excel(DATA_FILE, sheet_name='st_hydrogen', index_col=0, usecols='A:D')
st_other = pd.read_excel(DATA_FILE, sheet_name='st_other', index_col=0, usecols='A:I')

C:\Users\harri\AppData\Local\Temp\ipykernel_8840\498229758.py:20: FutureWarning: Defining usecols with out of bounds indices is deprecated and will raise a ParserError in a future version.
  gen_gas = pd.read_excel(DATA_FILE, sheet_name='gen_gas', index_col=0, usecols='A:R')


In [7]:
if filter_operating_ff_plants:
    gen_gas = gen_gas[gen_gas['status']=='operating']
    gen_gas_chp = gen_gas_chp[gen_gas_chp['status']=='operating']
    gen_coal = gen_coal[gen_coal['status']=='operational']
    gen_coal_chp = gen_coal_chp[gen_coal_chp['status']=='operational']

In [8]:
if add_gas_cold_start:
    gen_gas = grouped_gas_ramping(gen_gas)

In [9]:
# create chp timeseries based on country temperature profile
gas_chp_timeseries=chp_unit_profile(chp, gen_gas_chp)
coal_chp_timeseries=chp_unit_profile(chp, gen_coal_chp)
oil_chp_timeseries=chp_unit_profile(chp, gen_oil_chp)
res_chp_timeseries=chp_unit_profile(chp, gen_res_chp)

In [10]:
# create nuclear time series to replicate maintenance profile
nuclear_p_max_time_series, nuclear_p_min_time_series = apply_nuclear_outages(chp, gen_nuclear, nuclear_p_min=P_MIN_NUCLEAR, french_nucl_cf=0.85, other_nucl_cf=0.95)

In [11]:
network = pypsa.Network()
network.set_snapshots(load.index)

network.import_components_from_dataframe(buses, 'Bus')
network.import_components_from_dataframe(cbf_buses, 'Bus')
network.import_components_from_dataframe(buses_hydrogen, 'Bus')

network.import_components_from_dataframe(lines, 'Line')
network.lines['s_nom_extendable'] = True  

network.madd("Link", links.index,
             bus0=links['bus0'].tolist(), bus1= links['bus1'].tolist(),
             p_nom=links['p_nom'].tolist(), p_max_pu=links['p_max_pu'].to_list())

network.madd("Generator", gen_cbf.index, suffix='_CBF', carrier='CBF',
             bus=gen_cbf.index.tolist(), p_nom=gen_cbf['p_nom'].to_list(), p_nom_extendable=False, 
             marginal_cost=gen_cbf['marginal_cost'].to_list())

dict_carriers = {
    'Lignite': 0.334, 'Hard coal': 0.354, 'Gas': 0.187, 'Gas CCS': 0, 'Oil': 0.334, 'Hydrogen': 0, 'Biomass': 0.403, 'Biogas': 0.178, 'BECCS': 0,
    'Geothermal': 0.026, 'Wind': 0, 'Wind offshore': 0, 'PV': 0, 'HPS': 0, 'Hydro': 0, 'Other RES': 0, 'CBF': 0, 'VOLL': 0, 'Battery': 0, 'Other storage' : 0, 'Nuclear': 0, 'Electrolysis':0, 'Hydrogen storage':0, 'Hydrogen for power':0
}

for carrier in dict_carriers:
    network.add("Carrier", name=carrier, co2_emissions=dict_carriers[carrier])

network.madd("Load", load.columns, bus=load.columns, p_set=load)
if hydrogen_demand:
    network.madd("Load", load_hydrogen.columns, bus=load_hydrogen.columns, p_set=load_hydrogen, carrier='Hydrogen')

In [12]:
network.madd('Generator', 
      gen_pv['bus'],
      suffix='_PV',
      bus=gen_pv['bus'].to_list(), 
      p_nom_extendable=False, 
      p_nom=gen_pv['p_nom'].to_list(),
      carrier='PV', 
      marginal_cost=gen_pv['marginal_cost'].to_list(), 
      p_max_pu=pv)

network.madd('Generator', 
      gen_wind['bus'],
      suffix='_Wind',
      bus=gen_wind['bus'].to_list(), 
      p_nom_extendable=False, 
      p_nom=gen_wind['p_nom'].to_list(), 
      carrier='Wind', 
      marginal_cost=gen_wind['marginal_cost'].to_list(), 
      p_max_pu=wind)

network.madd('Generator', 
      gen_wind_offshore['bus'],
      suffix='_Wind_offshore',
      bus=gen_wind_offshore['bus'].to_list(), 
      p_nom_extendable=False, 
      p_nom=gen_wind_offshore['p_nom'].to_list(), 
      carrier='Wind offshore', 
      marginal_cost=gen_wind_offshore['marginal_cost'].to_list(), 
      p_max_pu=wind_offshore)

if add_gas_cold_start:
    network.madd('Generator', 
          gen_gas.index,
          bus=gen_gas['bus'].to_list(), 
          p_nom_extendable=False, 
          p_nom=gen_gas['p_nom'].to_list(), 
          carrier=gen_gas['carrier'].to_list(), 
          marginal_cost=gen_gas['marginal_cost'].to_list(),
          efficiency=gen_gas['efficiency'].to_list(),
          committable=True,
          min_up_time=gen_gas['min_up_time'].to_list(),
          )
else:
    network.madd('Generator', 
          gen_gas.index,
          bus=gen_gas['bus'].to_list(), 
          p_nom_extendable=False, 
          p_nom=gen_gas['p_nom'].to_list(), 
          carrier=gen_gas['carrier'].to_list(), 
          marginal_cost=gen_gas['marginal_cost'].to_list(),
          efficiency=gen_gas['efficiency'].to_list()
          )

network.madd('Generator', 
      gen_oil.index,
      bus=gen_oil['bus'].to_list(), 
      p_nom_extendable=False, 
      p_nom=gen_oil['p_nom'].to_list(), 
      carrier='Oil', 
      marginal_cost=gen_oil['marginal_cost'].to_list(),
      efficiency=gen_oil['efficiency'].to_list()
      )

network.madd('Generator', 
      gen_coal.index,
      bus=gen_coal['bus'].to_list(), 
      p_nom_extendable=False, 
      p_nom=gen_coal['p_nom'].to_list(), 
      carrier=gen_coal['carrier'].to_list(), 
      marginal_cost=gen_coal['marginal_cost'].to_list(),
      efficiency=gen_coal['efficiency'].to_list()
      )

network.madd('Generator', 
      gen_nuclear.index,
      bus=gen_nuclear['bus'].to_list(), 
      p_nom_extendable=False, 
      p_nom=gen_nuclear['p_nom'].to_list(), 
      carrier='Nuclear', 
      marginal_cost=gen_nuclear['marginal_cost'].to_list(),
      efficiency=gen_nuclear['efficiency'].to_list(),
      p_max_pu=nuclear_p_max_time_series,
      p_min_pu=nuclear_p_min_time_series
      )

network.madd('Generator', 
      gen_biomass.index,
      bus=gen_biomass['bus'].to_list(), 
      p_nom_extendable=False, 
      p_nom=gen_biomass['p_nom'].to_list(), 
      carrier=gen_biomass['carrier'].to_list(), 
      marginal_cost=gen_biomass['marginal_cost'].to_list(),
      efficiency=gen_biomass['efficiency'].to_list(),
      p_max_pu=gen_biomass['p_max_pu'].to_list()
      )

network.madd('Generator', 
      gen_biogas['bus'],
      suffix='_Biogas',
      bus=gen_biogas['bus'].to_list(), 
      p_nom_extendable=False, 
      p_nom=gen_biogas['p_nom'].to_list(), 
      carrier='Biogas', 
      marginal_cost=gen_biogas['marginal_cost'].to_list(),
      efficiency=gen_biogas['efficiency'].to_list(),
      p_max_pu=gen_biogas['p_max_pu'].to_list()
      )

network.madd('Generator', 
       gen_ror.index,
       bus=gen_ror['bus'].to_list(), 
       p_nom_extendable=False, 
       p_nom=gen_ror['p_nom'].to_list(), 
       carrier=gen_ror['carrier'].to_list(),
       marginal_cost=gen_ror['marginal_cost'].to_list(),
       p_max_pu=ror
       )

network.madd('Generator', 
       gen_reservoir.index,
       bus=gen_reservoir['bus'].to_list(), 
       p_nom_extendable=False, 
       p_nom=gen_reservoir['p_nom'].to_list(), 
       carrier=gen_reservoir['carrier'].to_list(),
       marginal_cost=gen_reservoir['marginal_cost'].to_list(),
       p_max_pu=inflow
       )

network.madd('Generator', 
      gen_other_res['bus'],
      suffix='_OtherRES',
      bus=gen_other_res['bus'].to_list(), 
      p_nom_extendable=False, 
      p_nom=gen_other_res['p_nom'].to_list(), 
      carrier='Other RES', 
      marginal_cost=gen_other_res['marginal_cost'].to_list(),
      p_max_pu=gen_other_res['p_max_pu'].to_list()
      )

network.madd('Generator', 
      gen_dsr['bus'],
      suffix='_DSR',
      bus=gen_dsr['bus'].to_list(), 
      p_nom_extendable=False, 
      p_nom=50000,#gen_dsr['p_nom'].to_list(), 
      carrier='DSR', 
      marginal_cost=gen_dsr['marginal_cost'].to_list(),
      p_max_pu=gen_dsr['p_max_pu'].to_list()
      )

network.madd('Generator', 
      gen_gas_chp.index,
      bus=gen_gas_chp['bus'].to_list(), 
      p_nom_extendable=False, 
      p_nom=gen_gas_chp['p_nom'].to_list(), 
      carrier=gen_gas_chp['carrier'].to_list(), 
      marginal_cost=gen_gas_chp['marginal_cost'].to_list(),
      p_max_pu=1.32*gas_chp_timeseries,
      p_min_pu=gas_chp_timeseries,
      efficiency=gen_gas_chp['efficiency'].to_list()
      )

network.madd('Generator', 
      gen_coal_chp.index,
      bus=gen_coal_chp['bus'].to_list(), 
      p_nom_extendable=False, 
      p_nom=gen_coal_chp['p_nom'].to_list(), 
      carrier=gen_coal_chp['carrier'].to_list(), 
      marginal_cost=gen_coal_chp['marginal_cost'].to_list(),
      p_max_pu=coal_chp_timeseries,
      p_min_pu=0.9*coal_chp_timeseries,
      efficiency=gen_coal_chp['efficiency'].to_list()
      )

network.madd('Generator', 
      gen_oil_chp.index,
      bus=gen_oil_chp['bus'].to_list(), 
      p_nom_extendable=False, 
      p_nom=gen_oil_chp['p_nom'].to_list(), 
      carrier=gen_oil_chp['carrier'].to_list(), 
      marginal_cost=gen_oil_chp['marginal_cost'].to_list(),
      p_max_pu=1.32*oil_chp_timeseries,
      p_min_pu=oil_chp_timeseries,
      efficiency=gen_oil_chp['efficiency'].to_list()
      )

network.madd('Generator', 
      gen_res_chp.index,
      bus=gen_res_chp['bus'].to_list(), 
      p_nom_extendable=False, 
      p_nom=gen_res_chp['p_nom'].to_list(), 
      carrier=gen_res_chp['carrier'].to_list(), 
      marginal_cost=gen_res_chp['marginal_cost'].to_list(),
      p_max_pu=1.32*res_chp_timeseries,
      p_min_pu=res_chp_timeseries,
      efficiency=gen_res_chp['efficiency'].to_list()
      )

network.madd('Generator', 
      gen_bio_chp.index,
      bus=gen_bio_chp['bus'].to_list(), 
      p_nom_extendable=False, 
      p_nom=gen_bio_chp['p_nom'].to_list(), 
      carrier=gen_bio_chp['carrier'].to_list(), 
      marginal_cost=gen_bio_chp['marginal_cost'].to_list(),
      p_max_pu=chp_bio,
      p_min_pu=chp_bio*0.9,
      efficiency=gen_bio_chp['efficiency'].to_list()
      )

network.madd("StorageUnit", st_hps.index, bus=st_hps['bus'].tolist(), carrier=st_hps['carrier'].tolist(),
             p_nom=st_hps['p_nom'].tolist(), p_nom_extendable=False, max_hours=st_hps['max_hours'].to_list(),
             p_max_pu=st_hps['p_max_pu'].tolist(),
             efficiency_dispatch=st_hps['efficiency_dispatch'].tolist(),
             standing_loss=st_hps['standing_loss'].tolist())
            
network.madd("StorageUnit", st_battery.index, bus=st_battery['bus'].tolist(), carrier=st_battery['carrier'].tolist(),
             p_nom=st_battery['p_nom'].tolist(), p_nom_extendable=False, max_hours=st_battery['max_hours'].to_list(),
             p_max_pu=st_battery['p_max_pu'].tolist(),
             efficiency_dispatch=st_battery['efficiency_dispatch'].tolist(),
             standing_loss=st_battery['standing_loss'].tolist())

network.madd("StorageUnit", st_other.index, bus=st_other['bus'].tolist(), carrier=st_other['carrier'].tolist(),
             p_nom=st_other['p_nom'].tolist(), p_nom_extendable=False, max_hours=st_other['max_hours'].to_list(),
             p_max_pu=st_other['p_max_pu'].tolist(),
             efficiency_dispatch=st_other['efficiency_dispatch'].tolist(),
             standing_loss=st_other['standing_loss'].tolist())

network.madd("Link", links_electrolysis.index,
             bus0=links_electrolysis['bus0'].tolist(), bus1= links_electrolysis['bus1'].tolist(),
             p_nom=links_electrolysis['p_nom'].tolist(), p_nom_extendable=links_electrolysis['p_nom_extendable'].tolist(), carrier=links_electrolysis['carrier'].tolist(), 
             efficiency=links_electrolysis['efficiency'].tolist())

network.madd("Store", st_hydrogen.index, bus=st_hydrogen['bus'].tolist(), carrier=st_hydrogen['carrier'].tolist(),
             e_nom=st_hydrogen['e_nom'].tolist(), e_nom_extendable=False, e_cyclic=True)

Index(['EN_NorthEast_H2_Storage', 'EN_NorthWest_H2_Storage',
       'EN_Yorkshire_H2_Storage', 'EN_EastMidlands_H2_Storage',
       'EN_WestMidlands_H2_Storage', 'EN_East_H2_Storage',
       'EN_London_H2_Storage', 'EN_SouthEast_H2_Storage',
       'EN_SouthWest_H2_Storage', 'EN_Wales_H2_Storage', 'Scotland_H2_Storage',
       'NorthernIreland_H2_Storage', 'Austria_H2_Storage',
       'Belgium_H2_Storage', 'Bulgaria_H2_Storage', 'Croatia_H2_Storage',
       'Czechia_H2_Storage', 'Denmark_H2_Storage', 'Estonia_H2_Storage',
       'Finland_H2_Storage', 'France_H2_Storage', 'Germany_H2_Storage',
       'Greece_H2_Storage', 'Hungary_H2_Storage', 'Ireland_H2_Storage',
       'Italy_H2_Storage', 'Latvia_H2_Storage', 'Lithuania_H2_Storage',
       'Netherlands_H2_Storage', 'Poland_H2_Storage', 'Portugal_H2_Storage',
       'Romania_H2_Storage', 'Slovakia_H2_Storage', 'Slovenia_H2_Storage',
       'Spain_H2_Storage', 'Sweden_H2_Storage'],
      dtype='object', name='name')

In [11]:
network.optimize(solver_name='highs')

C:\Users\harri\Anaconda3\envs\pypsa-310\lib\site-packages\numpy\core\numeric.py:407: RuntimeWarning: invalid value encountered in cast
  multiarray.copyto(res, fill_value, casting='unsafe')
C:\Users\harri\Anaconda3\envs\pypsa-310\lib\site-packages\numpy\core\numeric.py:407: RuntimeWarning: invalid value encountered in cast
  multiarray.copyto(res, fill_value, casting='unsafe')
C:\Users\harri\Anaconda3\envs\pypsa-310\lib\site-packages\numpy\core\numeric.py:407: RuntimeWarning: invalid value encountered in cast
  multiarray.copyto(res, fill_value, casting='unsafe')
C:\Users\harri\Anaconda3\envs\pypsa-310\lib\site-packages\numpy\core\numeric.py:407: RuntimeWarning: invalid value encountered in cast
  multiarray.copyto(res, fill_value, casting='unsafe')
INFO:linopy.model: Solve linear problem using Highs solver
INFO:linopy.io:Writing objective.
Writing variables.: 100%|██████████████████████████████████████████████████████████████| 10/10 [00:54<00:00,  5.41s/it]
INFO:linopy.io: Writing tim

('ok', 'optimal')

In [12]:
p_by_carrier = network.generators_t.p.groupby(network.generators.carrier, axis=1).sum()
generation_by_fuel_yearly = p_by_carrier.sum()/1000000
generation_by_fuel_yearly

carrier
BECCS              0.000000
Biogas           103.153780
Biomass          132.642449
CBF                0.687989
DSR                1.937272
Gas              416.399118
Hard coal         50.238175
Hydro            293.832866
Lignite           26.725573
Nuclear          589.725310
Oil               29.020249
Other RES         68.625221
PV               724.420384
Wind             870.779160
Wind offshore    480.426763
dtype: float64

In [13]:
prices = network.buses_t.marginal_price
prices.to_csv('prices.csv')

In [14]:
p_by_carrier = network.generators_t.p.groupby([network.generators.carrier, network.generators.bus], axis=1).sum()
generation_by_fuel_by_country = p_by_carrier.sum()/1000000
generation_by_fuel_by_country.to_csv('generation_by_fuel_by_country.csv')

In [15]:
p_by_unit = network.generators_t.p.sum()/1000000
generation_by_unit = p_by_unit
generation_by_unit.to_csv('generation_by_unit.csv')

In [16]:
generation_by_link = network.links_t.p0.sum()/1000000
country_from = ['UK' if c == 'EN' else 'UK' if c == 'Sc' else 'UK' if c == 'No' else c for c in generation_by_link.index.str[5:7]]
country_to = ['UK' if c.islower() else c for c in generation_by_link.index.str[-2:]]
generation_by_link = generation_by_link.to_frame(name='TWh')
generation_by_link['from'] = country_from
generation_by_link['to'] = country_to
generation_by_link.to_csv('generation_by_link.csv')

In [17]:
generation_by_link = network.links_t.p0
generation_by_link.to_csv('generation_by_link_hourly.csv')

In [18]:
generation_by_line = network.lines_t.p0.sum()/1000000
generation_by_line.to_csv('generation_by_line.csv')

In [19]:
original_line_capacity = network.lines.s_nom
optimized_line_capacity = network.lines.s_nom_opt
optimized_line_capacity.to_csv('line_s_nom_opt.csv')

In [20]:
domestic_generation_by_fuel_yearly = generation_by_fuel_yearly
domestic_generation_by_fuel_yearly=domestic_generation_by_fuel_yearly.drop('CBF')
generation_by_fuel_yearly_percentage = domestic_generation_by_fuel_yearly.div(domestic_generation_by_fuel_yearly.sum(), axis=0)
generation_by_fuel_yearly_percentage = generation_by_fuel_yearly_percentage.round(4)
generation_by_fuel_yearly_percentage = (generation_by_fuel_yearly_percentage * 100).astype(str) + '%'
generation_by_fuel_yearly_percentage

carrier
BECCS                           0.0%
Biogas           2.7199999999999998%
Biomass          3.5000000000000004%
DSR                            0.05%
Gas                           10.99%
Hard coal        1.3299999999999998%
Hydro                          7.76%
Lignite          0.7100000000000001%
Nuclear                       15.57%
Oil                            0.77%
Other RES                      1.81%
PV                            19.12%
Wind                          22.99%
Wind offshore                 12.68%
dtype: object

In [21]:
capacity_by_fuel = network.generators.groupby("carrier")["p_nom"].sum()
capacity_by_fuel

carrier
BECCS            0.000000e+00
Biogas           1.374849e+04
Biomass          2.089929e+04
CBF              2.000000e+05
DSR              1.800000e+06
Gas              2.114838e+05
Hard coal        3.516500e+04
Hydro            1.105667e+05
Lignite          2.374300e+04
Nuclear          9.572400e+04
Oil              2.784170e+04
Other RES        1.852238e+04
PV               6.385830e+05
Wind             3.926902e+05
Wind offshore    1.343640e+05
Name: p_nom, dtype: float64

In [22]:
buses_UK = list(set([b for b in network.generators.bus.to_list() if 'EN_' in b])) + ['NorthernIreland', 'Scotland']
generation_hourly_uk_by_fuel = p_by_carrier.iloc[:, p_by_carrier.columns.get_level_values(1).isin(buses_UK)].groupby(['carrier'], axis=1).sum()/1000000
generation_hourly_uk_by_fuel.to_csv("generation_hourly_uk_by_fuel.csv")

In [23]:
storage_capacity = network.storage_units.groupby("carrier")["p_nom"].sum()
storage_capacity

carrier
Battery          99449.000001
HPS              56455.000000
Other storage      512.383140
Name: p_nom, dtype: float64

In [24]:
generation_carriers = pd.merge(network.generators, network.carriers, left_on="carrier", right_index=True)
emissions = network.generators_t.p * generation_carriers["co2_emissions"] / generation_carriers["efficiency"]

(emissions.sum().sum()*1000000000000/1000000)/(generation_by_fuel_yearly.sum()*1000000000)

126.92552792598006

In [25]:
emissions.sum().sum()/1000000

480.87187143293716

In [26]:
380*1000/(61.4*24*365)

0.7064982969672631

In [27]:
output = network.stores_t.p
state_of_charge = network.stores_t.e
output.to_csv('store_output.csv')
state_of_charge.to_csv('store_state_of_charge.csv')

In [28]:
network.storage_units_t.keys()

dict_keys(['p_min_pu', 'p_max_pu', 'p_set', 'q_set', 'marginal_cost', 'state_of_charge_set', 'inflow', 'p', 'p_dispatch', 'p_store', 'q', 'state_of_charge', 'spill', 'mu_upper', 'mu_lower', 'mu_state_of_charge_set'])

In [29]:
network.export_to_netcdf(path="network.nc")

INFO:pypsa.io:Exported network network.nc has links, generators, storage_units, carriers, buses, lines, loads, stores


<xarray.Dataset>
Dimensions:                            (snapshots: 8760, investment_periods: 0,
                                        links_i: 156, links_t_p0_i: 127,
                                        links_t_p1_i: 127,
                                        links_t_mu_lower_i: 150,
                                        links_t_mu_upper_i: 124,
                                        generators_i: 1835,
                                        ...
                                        lines_t_p1_i: 17, loads_i: 72,
                                        loads_t_p_set_i: 47, loads_t_p_i: 47,
                                        stores_i: 36, stores_t_p_i: 11,
                                        stores_t_e_i: 11,
                                        stores_t_mu_upper_i: 11,
                                        stores_t_mu_lower_i: 11)
Coordinates: (12/36)
  * snapshots                          (snapshots) int64 0 1 2 ... 8758 8759
  * investment_periods                 (investment_periods) object 
  * links_i                            (links_i) object 'link_FR_EN_SouthEast...
  * links_t_p0_i                       (links_t_p0_i) object 'link_FR_EN_Sout...
  * links_t_p1_i                       (links_t_p1_i) object 'link_FR_EN_Sout...
  * links_t_mu_lower_i                 (links_t_mu_lower_i) object 'link_FR_E...
    ...                                 ...
  * loads_t_p_i                        (loads_t_p_i) object 'EN_NorthEast' .....
  * stores_i                           (stores_i) object 'EN_NorthEast_H2_Sto...
  * stores_t_p_i                       (stores_t_p_i) object 'EN_NorthEast_H2...
  * stores_t_e_i                       (stores_t_e_i) object 'EN_NorthEast_H2...
  * stores_t_mu_upper_i                (stores_t_mu_upper_i) object 'EN_North...
  * stores_t_mu_lower_i                (stores_t_mu_lower_i) object 'EN_North...
Data variables: (12/91)
    snapshots_snapshot                 (snapshots) datetime64[ns] 2030-01-01 ...
    snapshots_objective                (snapshots) float64 1.0 1.0 ... 1.0 1.0
    snapshots_stores                   (snapshots) float64 1.0 1.0 ... 1.0 1.0
    snapshots_generators               (snapshots) float64 1.0 1.0 ... 1.0 1.0
    investment_periods_objective       (investment_periods) object 
    investment_periods_years           (investment_periods) object 
    ...                                 ...
    stores_build_year                  (stores_i) int64 0 0 0 0 0 ... 0 0 0 0 0
    stores_e_nom_opt                   (stores_i) float64 1.997e+03 ... 0.0
    stores_t_p                         (snapshots, stores_t_p_i) float64 10.3...
    stores_t_e                         (snapshots, stores_t_e_i) float64 901....
    stores_t_mu_upper                  (snapshots, stores_t_mu_upper_i) float64 ...
    stores_t_mu_lower                  (snapshots, stores_t_mu_lower_i) float64 ...
Attributes:
    network__linearized_uc:  0
    network__multi_invest:   0
    network_name:            
    network_objective:       74625336831.47462
    network_pypsa_version:   0.22.1
    network_srid:            4326
    meta:                    {}